Reference Link : https://huggingface.co/t5-base

Hugging Face T5 Documentation: <br>
    https://huggingface.co/docs/transformers/model_doc/t5#transformers.T5Model 

<b>Problem Statement </b>: Fine tune a model to identify the product given a product review.<br>
<b>Data set</b> https://data.world/datafiniti/consumer-reviews-of-amazon-products 

In [1]:
#!pip install datasets transformers rouge-score nltk

     -------------------------------------- 486.2/486.2 kB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 110.5/110.5 kB 6.7 MB/s eta 0:00:00
     -------------------------------------- 132.0/132.0 kB 7.6 MB/s eta 0:00:00
     --------------------------------------- 21.5/21.5 MB 24.2 MB/s eta 0:00:00
     ------------------------------------- 163.8/163.8 kB 10.2 MB/s eta 0:00:00
     ---------------------------------------- 77.1/77.1 kB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 24.5 MB/s eta 0:00:00
     ------------------------------------- 236.8/236.8 kB 14.2 MB/s eta 0:00:00
     -------------------------------------- 263.9/263.9 kB 8.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=703c0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.1.0 requires spacy<3.2.0,>=3.1.0, but you have spacy 3.3.1 which is incompatible.


In [10]:
import pandas as pd

In [15]:
path  = "D:/Sups/Python/OpenAI/T5/review_data_Product.csv"
df = pd.read_csv(path)
df.head()

,review,prod_sentiment
0,Solid entry level Kindle. Great for kids. Gift...,"Product:Kindle , Sentiment:Positive"
1,This make an excellent ebook reader. Don't exp...,"Product:Ebook Reader , Sentiment:Positive"
2,I use this every day on my commute. Great batt...,"Product:Unknown , Sentiment:Positive"
3,Everything is great except that i can't read e...,"Product:Kindle Books , Sentiment:Positive"
4,I never used kindle before. My wife gifted me ...,"Product:Kindle\n , Sentiment:Positive"


In [18]:
df.head()

,review,prod_sentiment
0,Solid entry level Kindle. Great for kids. Gift...,"Product:Kindle , Sentiment:Positive"
1,This make an excellent ebook reader. Don't exp...,"Product:Ebook Reader , Sentiment:Positive"
2,I use this every day on my commute. Great batt...,"Product:Unknown , Sentiment:Positive"
3,Everything is great except that i can't read e...,"Product:Kindle Books , Sentiment:Positive"
4,I never used kindle before. My wife gifted me ...,"Product:Kindle\n , Sentiment:Positive"


We will use one of the T5 models to identify the product, sentiment and main concern area of the cusomer from the review

In [3]:
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install rich[jupyter]

In [4]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

In [5]:
# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [22]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [23]:
print(device)

cpu


In [24]:
class ProductExtractor(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }


In [25]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 10 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [26]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals


In [27]:
def T5Trainer(
    dataframe, source_text, target_text, model_params, output_dir="./review_outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = ProductExtractor(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = ProductExtractor(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions_prod_senti.csv"))

    console.save_text(os.path.join(output_dir, "review_logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "review_model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'review_predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'review_logs.txt')}\n""")


In [28]:

# let's define model parameters specific to T5
model_params = {
    "MODEL": "t5-base",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TRAIN_EPOCHS": 3,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 50,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}


In [29]:
# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)


In [30]:
# T5 accepts prefix of the task to be performed:
# Since we are summarizing, let's add summarize to source text as a prefix
df["Review"] = "Extract Product: " + df["Review"]

T5Trainer(
    dataframe=df[:500], #INPUT FIRST 500 ROWS ONLY.
    source_text="Review",
    target_text="Product",
    model_params=model_params,
    output_dir="review_outputs",
)


[17:29:20] [Model]: Loading t5-base...                                           <ipython-input-27-7a5edd4898d1>:16
                                                                                                                   

C:\Users\supriya.aras\Anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[17:29:27] [Data]: Reading data...                                               <ipython-input-27-7a5edd4898d1>:27
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                             source_text                              |                target_text               |
|----------------------------------------------------------------------+------------------------------------------|
| Extract Product and Sentiment: Solid entry level Kindle. Great for   |    Product:Kindle , Sentiment:Positive   |
| kids. Gifted for a kid of my friend and they love to use it to read  |                                          |
|                       more than their iPads.                         |                                          |
| Extract Product and Sentiment: This make an excellent ebook reader.  | Product:Ebook Reader , Sentiment:Positive|
| Don't expect much from this device except to read basic ebooks. The  |                                          |
|        good thing is it's cheap and good to read in the sun.         |                                          |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (500, 2)

...

Completed 10

[20:27:53] [Validation Completed.]                                              <ipython-input-27-7a5edd4898d1>:105
                                                                                                                   

[Model] Model saved @ review_outputs\review_model_files

[Validation] Generation on Validation data saved @ review_outputs\review_predictions.csv

[Logs] Logs saved @ review_outputs\review_logs.txt

In [61]:
df[:500].shape

(500, 2)

In [ ]:
'''# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom")'''

In [18]:
tokenizer = T5Tokenizer.from_pretrained("./review_outputs/model_files",local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained("./review_outputs/model_files",local_files_only=True)

ids = tokenizer("Extract Product :This laptop has good battery life. \
It works quite fast i must say", return_tensors="pt").input_ids

generated_ids = model.generate(
              input_ids = ids,
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
print(preds)
       

[':Positive :Unknown, Extract :Positive']


In [ ]:

input_ids = tokenizer("translate from English to Hindi: The house is wonderful. It is painted blue", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))